In [1]:
import numpy as np
import math
import pickle
from openpyxl import Workbook
from openpyxl import load_workbook
import mojimoji
import numpy as np
import glob
import re
import MeCab
import types
import pandas as pd
from collections import Counter
%matplotlib inline

mc = MeCab.Tagger("-Ochasen")

In [2]:
dirpath = "data/excel/"

filename = "A_W_171116.xlsx"
A_W_171116_excel = pd.ExcelFile(dirpath + filename)
sheet_name = A_W_171116_excel.sheet_names[0]
A_W_171116_df = A_W_171116_excel.parse(sheet_name)

filename = "A_W_171128.xlsx"
A_W_171128_excel = pd.ExcelFile(dirpath + filename)
sheet_name = A_W_171128_excel.sheet_names[0]
A_W_171128_df = A_W_171128_excel.parse(sheet_name)

filename = "171116.xlsx"
S_H_171116_excel = pd.ExcelFile(dirpath + filename)
sheet_name = S_H_171116_excel.sheet_names[0]
S_H_171116_df = S_H_171116_excel.parse(sheet_name)

filename = "2015斉藤.xlsx"
S_2015_excel = pd.ExcelFile(dirpath + filename)
sheet_name = S_2015_excel.sheet_names[0]
S_2015_df = S_2015_excel.parse(sheet_name)

## 必要なデータ
- id
    - A_W_YYMMDDのNo.(0列目)
    - SH_171116のid(0列目)
    - 斉藤2015の発言番号(0列目)
- reply
    - A_W_YYMMDDの返信元(1列目)
    - SH_171116のreply(3列目)
    - 斉藤2015の返信元(1列目)
- group
    - A_W_YYMMDDのgid(6列目)
    - SH_171116のgid(5列目)
    - 斉藤2015のグループ番号(6列目)
- who
    - A_W_YYMMDDのニックネーム(8列目)
    - SH_171116のcname(6列目)
    - 斉藤2015のニックネーム(8列目)
- body
    - A_W_YYMMDDの発言内容(11列目)
    - SH_171116のbody(8列目)
    - 斉藤2015の発言内容(11列目)
- epistemic_a
    - A_W_YYMMDDのEpistemic(A)(12列目)
    - Epistemic(H)(9列目)
- epistemic_b
    - A_W_YYMMDDのWatanabe(13列目)
    - Epistemic(S)(10列目)
- coordination_a
    - A_W_YYMMDDのCoordination(A)(17列目)
    - SH_171116のCoordination(H)(13列目)
- coordination_b
    - A_W_YYMMDDのWatanabe.1(18列目)
    - SH_171116のCoordination(S)(14列目)
    
## 各dfで必要な列
- A_W_YYMMDD[0, 1, 6, 8, 11, 12, 13, 17, 18]
- SH_171116[0, 3, 5, 6, 8, 9. 10 13, 14]
- 斉藤2015[0, 1, 6, 8, 11]

In [3]:
A_W_YYMMDD_use = [0, 1, 6, 8, 11, 12, 13, 17, 18]
S_H_171116_use = [0, 3, 5, 6, 8, 9, 10, 13, 14]
S_2015_use = [0, 1, 6, 8, 11]

In [4]:
# A_W_YYMMDDの列名の統合
A_W_171116_df.columns = A_W_171128_df.columns

delete_columns_list = []
for i, column in enumerate(A_W_171116_df):
    if not i in A_W_YYMMDD_use:
        delete_columns_list.append(column)
        
A_W_171116_df = A_W_171116_df.drop(delete_columns_list, axis=1)
A_W_171128_df = A_W_171128_df.drop(delete_columns_list, axis=1)

delete_columns_list = []
for i, column in enumerate(S_H_171116_df):
    if not i in S_H_171116_use:
        delete_columns_list.append(column)
S_H_171116_df = S_H_171116_df.drop(delete_columns_list, axis=1)

delete_columns_list = []
for i, column in enumerate(S_2015_df):
    if not i in S_2015_use:
        delete_columns_list.append(column)
S_2015_df = S_2015_df.drop(delete_columns_list, axis=1)

In [5]:
print(not False in S_H_171116_df['id'].values == S_2015_df['発言番号'].values)
print(not False in S_H_171116_df['cname'].values == S_2015_df['ニックネーム'].values)
print(not False in S_H_171116_df['body'].values == S_2015_df['発言内容'].values)

True
True
True


In [6]:
S_H_171116_df["reply"] = S_2015_df['返信元']
S_H_171116_df["gid"] = S_2015_df['グループ番号']
concat_border = 299
A_W_171116_df = A_W_171116_df[0:concat_border]
A_W_171128_df = A_W_171128_df[concat_border:]
A_W_df = pd.concat([A_W_171116_df, A_W_171128_df])

In [7]:
# 要素含まれるNanの数を確認
A_W_df.isnull().sum()

No.                   3
返信元                   3
gid                   3
ニックネーム                3
発言内容                  3
Epistemic(A)          3
Watanabe              3
Coordination(A)    3603
Watanabe.1         4294
dtype: int64

# 重要！！

In [8]:
# A_W_dfの人間が付けたラベル列のNanをBlankに変換
A_W_df = A_W_df.fillna('Blank')
# 要素にNanが含まれる列を再確認
A_W_df.isnull().any()

No.                False
返信元                False
gid                False
ニックネーム             False
発言内容               False
Epistemic(A)       False
Watanabe           False
Coordination(A)    False
Watanabe.1         False
dtype: bool

In [9]:
# 要素に含まれるNanの数を確認
S_H_171116_df.isnull().sum()

id                    0
reply                 0
gid                   0
cname                 0
body                  0
Epistemic(H)          1
Epistemic(S)          4
Coordination(H)    3933
Coordination(S)    4188
dtype: int64

# 重要！！

In [10]:
# S_H_171116_dfの人間が付けたラベル列のNanを取り除く
S_H_171116_df = S_H_171116_df.fillna('Blank')
# 要素にNanが含まれる列を再確認
S_H_171116_df.isnull().any()

id                 False
reply              False
gid                False
cname              False
body               False
Epistemic(H)       False
Epistemic(S)       False
Coordination(H)    False
Coordination(S)    False
dtype: bool

# 重要！！

In [11]:
A_W_df = A_W_df.drop_duplicates(subset=['ニックネーム', '発言内容'])
S_H_171116_df = S_H_171116_df.drop_duplicates(subset=['cname', 'body'])

# 重要！！

In [12]:
# 表記ゆれの修正と確認
A_W_df['Coordination(A)'] = A_W_df['Coordination(A)'].str.lower()
counter = Counter(A_W_df['Coordination(A)'])
print(counter)
A_W_df['Watanabe.1'] = A_W_df['Watanabe.1'].str.lower()
counter = Counter(A_W_df['Watanabe.1'])
print(counter)

Counter({'blank': 3347, 'technical coordination': 766, 'proceedings': 261, 'task division': 160, 'time management': 54, 'quote': 16})
Counter({'blank': 3978, 'technical coordination': 306, 'proceedings': 208, 'task division': 75, 'time management': 24, 'quote': 13})


# 重要！！

In [13]:
# 表記ゆれの確認
S_H_171116_df['Coordination(S)'] = S_H_171116_df['Coordination(S)'].str.lower()
counter = Counter(S_H_171116_df['Coordination(S)'])
print(counter)
S_H_171116_df['Coordination(H)'] = S_H_171116_df['Coordination(H)'].str.lower()
counter = Counter(S_H_171116_df['Coordination(H)'])
print(counter)

Counter({'blank': 3911, 'technical coordination': 600, 'proceedings': 126, 'time management': 62, 'quote': 26, 'task division': 14})
Counter({'blank': 3663, 'technical coordination': 780, 'proceedings': 219, 'time management': 35, 'quote': 30, 'task division': 12})


In [14]:
# 表記ゆれの修正と確認
A_W_df['Epistemic(A)'] = A_W_df['Epistemic(A)'].str.lower()
counter = Counter(A_W_df['Epistemic(A)'])
print(counter)
A_W_df['Watanabe'] = A_W_df['Watanabe'].str.lower()
counter = Counter(A_W_df['Watanabe'])
print(counter)

Counter({'off task': 2392, 'on task': 2092, 'no sense': 119, 'blank': 1})
Counter({'off task': 2444, 'on task': 2148, 'no sense': 11, 'blank': 1})


In [15]:
# 表記ゆれの確認
S_H_171116_df['Epistemic(S)'] = S_H_171116_df['Epistemic(S)'].str.lower()
counter = Counter(S_H_171116_df['Epistemic(S)'])
print(counter)
S_H_171116_df['Epistemic(H)'] = S_H_171116_df['Epistemic(H)'].str.lower()
counter = Counter(S_H_171116_df['Epistemic(H)'])
print(counter)

Counter({'off task': 2546, 'on task': 2179, 'no sense': 10, 'blank': 4})
Counter({'off task': 2560, 'on task': 2165, 'no sense': 13, 'blank': 1})


In [16]:
A_W_df = A_W_df[A_W_df['Epistemic(A)'] != 'no sense']
A_W_df = A_W_df[A_W_df['Watanabe'] != 'no sense']
A_W_df = A_W_df[A_W_df['Epistemic(A)'] != 'blank']
A_W_df = A_W_df[A_W_df['Watanabe'] != 'blank']
print(Counter(A_W_df['Epistemic(A)']))
print(Counter(A_W_df['Watanabe']))

Counter({'off task': 2390, 'on task': 2091})
Counter({'off task': 2337, 'on task': 2144})


In [17]:
S_H_171116_df = S_H_171116_df[S_H_171116_df['Epistemic(H)'] != 'no sense']
S_H_171116_df = S_H_171116_df[S_H_171116_df['Epistemic(S)'] != 'no sense']
S_H_171116_df = S_H_171116_df[S_H_171116_df['Epistemic(H)'] != 'blank']
S_H_171116_df = S_H_171116_df[S_H_171116_df['Epistemic(S)'] != 'blank']
print(Counter(S_H_171116_df['Epistemic(H)']))
print(Counter(S_H_171116_df['Epistemic(S)']))

Counter({'off task': 2558, 'on task': 2158})
Counter({'off task': 2538, 'on task': 2178})


# 重要！！

In [18]:
A_W_match_df = A_W_df[A_W_df['Coordination(A)'] == A_W_df['Watanabe.1']].reset_index(drop=True)
A_W_match_df.head()

,No.,返信元,gid,ニックネーム,発言内容,Epistemic(A),Watanabe,Coordination(A),Watanabe.1
0,1,\N,1,まこぴす,よろしくお願いします！,off task,off task,blank,blank
1,31,\N,1,哲,よろしくお願いします,off task,off task,blank,blank
2,70,\N,1,仙波,名前なのが恥ずかしいです…\n\nよろしくお願いします！,off task,off task,blank,blank
3,119,\N,1,まこぴす,早速課題やっちゃいましょう！,off task,off task,proceedings,proceedings
4,163,\N,1,仙波,やっちゃいましょう\n\nmoodleはゴミです！,on task,on task,blank,blank


# 重要！！

In [19]:
S_H_match_df = S_H_171116_df[S_H_171116_df['Coordination(H)'] == S_H_171116_df['Coordination(S)']].reset_index(drop=True)
S_H_match_df.head()

,id,reply,gid,cname,body,Epistemic(H),Epistemic(S),Coordination(H),Coordination(S)
0,1593,\N,1,世界のわたべ,よろしくお願いします。,off task,off task,blank,blank
1,1598,\N,1,ざきさん,よろしくです,off task,off task,blank,blank
2,1606,\N,1,あ,よろしくです,off task,off task,blank,blank
3,1659,\N,1,ざきさん,みなさんファイルをアップしましたか？,off task,off task,technical coordination,technical coordination
4,1704,\N,1,ざきさん,名前バレｗｗ,off task,off task,blank,blank


In [20]:
A_W_match_df.isnull().sum()

No.                0
返信元                0
gid                0
ニックネーム             0
発言内容               0
Epistemic(A)       0
Watanabe           0
Coordination(A)    0
Watanabe.1         0
dtype: int64

In [21]:
S_H_match_df.isnull().sum()

id                 0
reply              0
gid                0
cname              0
body               0
Epistemic(H)       0
Epistemic(S)       0
Coordination(H)    0
Coordination(S)    0
dtype: int64

# 重要！！

In [22]:
header = ['id', 'reply', 'group_id', 'cname', 'body', 'Epistemic_A', 'Epistemic_B', 'Coordination_A', 'Coordination_B']
A_W_df.columns = header
S_H_171116_df.columns = header
A_W_match_df.columns = header
S_H_match_df.columns = header
All_df = pd.concat([A_W_df, S_H_171116_df], ignore_index=True)
match_df = pd.concat([A_W_match_df, S_H_match_df], ignore_index=True)

print(A_W_df.shape)
print(S_H_171116_df.shape)
print(All_df.shape)
print(match_df.shape)

(4481, 9)
(4716, 9)
(9197, 9)
(7653, 9)


In [23]:
# データの形式の定義
data_format = ['id', 'reply', 'group_id', 'cname', 'body', 'epistemic', 'label']
# 名前からデータのindexに変換する辞書
data_index = {column : i for i, column in enumerate(data_format)}
print(data_index)

id = 0
reply = 1
group_id = 2
cname = 3
body = 4
epistemic = 5
label = 6

df_index = {column: i for i, column in enumerate(All_df)}
print(df_index)

{'group_id': 2, 'epistemic': 5, 'body': 4, 'label': 6, 'cname': 3, 'id': 0, 'reply': 1}
{'group_id': 2, 'Coordination_A': 7, 'body': 4, 'Coordination_B': 8, 'cname': 3, 'Epistemic_A': 5, 'id': 0, 'Epistemic_B': 6, 'reply': 1}


# 重要！！

In [24]:
df_use = [0, 1, 2, 3, 4]

# All_df_index['ラベル名']を変更して取り出すラベルを分ける
# epistemic_position = df_index['Epistemic_A']
# epistemic_position = df_index['Epistemic_B']
epistemic_position = df_index['Epistemic_A']
df_use.append(epistemic_position)

# label_position = df_index['Coordination_A']
# label_position = df_index['Coordination_B']
label_position = df_index['Coordination_A']
df_use.append(label_position)

print(df_use)

delete_columns_list = []
for i, column in enumerate(All_df):
#for i, column in enumerate(match_df):
    if not i in df_use:
        delete_columns_list.append(column)
input_df = All_df.drop(delete_columns_list, axis=1)
#input_df = match_df.drop(delete_columns_list, axis=1)
# input_df = A_W_consulted_df.drop(delete_columns_list, axis=1)
input_df.columns = data_format

[0, 1, 2, 3, 4, 5, 7]


In [25]:
input_df

,id,reply,group_id,cname,body,epistemic,label
0,1,\N,1,まこぴす,よろしくお願いします！,off task,blank
1,31,\N,1,哲,よろしくお願いします,off task,blank
2,70,\N,1,仙波,名前なのが恥ずかしいです…\n\nよろしくお願いします！,off task,blank
3,119,\N,1,まこぴす,早速課題やっちゃいましょう！,off task,proceedings
4,163,\N,1,仙波,やっちゃいましょう\n\nmoodleはゴミです！,on task,blank
5,194,\N,1,まこぴす,使いにくいです(笑),on task,blank
6,302,\N,1,哲,同意です,on task,blank
7,309,\N,1,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,on task,blank
8,385,\N,1,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,on task,blank
9,426,\N,1,まこぴす,自分の意見です！(笑),off task,blank


In [26]:
input_df = input_df[input_df['epistemic'] == 'off task'].reset_index(drop=True)
input_df

,id,reply,group_id,cname,body,epistemic,label
0,1,\N,1,まこぴす,よろしくお願いします！,off task,blank
1,31,\N,1,哲,よろしくお願いします,off task,blank
2,70,\N,1,仙波,名前なのが恥ずかしいです…\n\nよろしくお願いします！,off task,blank
3,119,\N,1,まこぴす,早速課題やっちゃいましょう！,off task,proceedings
4,426,\N,1,まこぴす,自分の意見です！(笑),off task,blank
5,813,\N,1,まこぴす,１度共有テキスト全部消して現在の回答案書きます！,off task,technical coordination
6,851,813,1,哲,ありがとうございます,off task,blank
7,888,\N,1,仙波,あざっす\n\nこれでいいのでは？,off task,blank
8,899,\N,1,まこぴす,こちらの回答案でよければ課題回答しちゃいますけど大丈夫ですか？,off task,proceedings
9,916,\N,1,仙波,自分は平気です,off task,blank


In [27]:
counter = Counter(input_df['label'])
print(counter)

Counter({'blank': 2621, 'technical coordination': 1543, 'proceedings': 479, 'task division': 171, 'time management': 88, 'quote': 46})


In [28]:
input_data = input_df.values

In [29]:
def preprocess(xs, trim_citation=False):
    zs = [""] * len(xs)
    '''全角空白'''
    for i, l in enumerate(xs):
        xs[i] = re.sub('　', ' ', str(l))
    '''多重引用'''
    for i, l in enumerate(xs):
        xs[i] = re.sub('＆ｇｔ；', '＞', l)
    '''引用部分は除外'''
    count_refer = 0
    for i, l in enumerate(xs):
        sen = ""
        have_refer = 1
        for x in l.split('\n'):
            if len(x) == 0:
                continue
            elif x[0].encode('utf-8') == '＞'.encode('utf-8'):
                have_refer = 1
                x = x[1:]
                if len(x) > 0 and x[0].encode('utf-8') != '＞'.encode('utf-8') and \
                        x[0:2].encode('utf-8') != ' ＞'.encode('utf-8') and x != ' ':
                    if trim_citation:
                        zs[i] += x + '\n'
                    else:
                        sen = sen + "＞ " + x + '\n'
                continue
            else:
                sen = sen + x + '\n'
        xs[i] = sen.strip()
        count_refer += have_refer
    '''顔文字'''
    pat = re.compile('（[^ぁ-んァ-ン一-龠]+?）', re.U)
    for i, l in enumerate(xs):
        xs[i] = re.sub(pat, ' KAOMOJI ', l)
    '''日本語及び記号{？！ー〜。、} のみ有効'''
    pat = re.compile('[^＞ Ａ-Ｚａ-ｚ０-９ぁ-んァ-ン一-龠？！ー〜。、\n]+?', re.U)
    for i, l in enumerate(xs):
        xs[i] = re.sub(pat, '', l)
    '''記号繰り返し'''
    pat_1 = re.compile('[ｗ]{1,}', re.U)
    pat_2 = re.compile('[？]{1,}', re.U)
    pat_3 = re.compile('[！]{1,}', re.U)
    pat_4 = re.compile('[～]{1,}', re.U)
    pat_5 = re.compile('[ー]{1,}', re.U)
    pat_6 = re.compile('[。]{1,}', re.U)

    for i, l in enumerate(xs):
        l = re.sub(pat_1, 'ｗ', l)
        l = re.sub(pat_2, '？', l)
        l = re.sub(pat_3, '！', l)
        l = re.sub(pat_4, '～', l)
        l = re.sub(pat_5, 'ー', l)
        l = re.sub(pat_6, '。', l)
        xs[i] = l

    if trim_citation:
        return zs
    else:
        return None

In [30]:
def make_xs(orig, trim_citation=False, use_seq2seq=False):

    tmp = np.copy(orig)
    xs = tmp[:, body]
    zs = preprocess(xs, trim_citation=(trim_citation or use_seq2seq))

    if use_seq2seq:
        tmp_prev = np.roll(tmp, 1, axis=0)
        tmp_prev[0, 0] = ""
        ts = tmp[:, [1, 2]]
        xs_prev = tmp_prev[:, body]

        for i in range(len(tmp)):
            if tmp[i, group_id] != tmp_prev[i, group_id]:  # group が異なる
                xs_prev[i] = ""
        preprocess(xs_prev, trim_citation=True)

        for i in range(len(xs)):
            replay_to = tmp[i, 2]
            if zs[i] != "":
                xs_prev[i] = zs[i]
            elif replay_to != -1:
                a = xs[:i][tmp[:i, 1] == replay_to]
                if len(a) != 0:
                    xs_prev[i] = a[-1]

        return xs, xs_prev
    else:
        return xs

In [31]:
def get_doc(xs):
    doc = []
    for ll in xs:
        sen = []
        for l in ll.split('\n'):
            for w in mc.parse(l).split('\n'):
                mx = w.split('\t')
                if len(mx[0]) > 0:
                    sen.append(mx[0])
        doc.append(sen)
    return doc

In [32]:
xs, xs_pre = make_xs(input_data, trim_citation=True, use_seq2seq=True)
doc = get_doc(xs)
doc_pre = get_doc(xs_pre)
for i in doc[:10]:
    print(" ".join(i))

よろしく お願い し ます ！ EOS
よろしく お願い し ます EOS
名前 な の が 恥ずかしい です EOS よろしく お願い し ます ！ EOS
早速 課題 やっ ちゃ い ましょ う ！ EOS
自分 の 意見 です ！ 笑 EOS
１ 度 共有 テキスト 全部 消し て 現在 の 回答 案 書き ます ！ EOS
ありがとう ござい ます EOS
あざ っす EOS これ で いい の で は ？ EOS
こちら の 回答 案 で よけれ ば 課題 回答 し ちゃい ます けど 大丈夫 です か ？ EOS
自分 は 平気 です EOS


In [33]:
wd_set = Counter([x for sen in doc + doc_pre for x in sen])
wd_ary = np.array(list(wd_set.keys()))
wd_cnt = np.array(list(wd_set.values()))
wd_ary = wd_ary[np.argsort(wd_cnt)[::-1]]
wd_cnt.sort()
wd_cnt = wd_cnt[::-1]
wd_to_id = {wd: i for i, wd in enumerate(wd_ary)}
lb_to_id = {lb: i for i, lb in enumerate({x for x in input_data[:, label]})}
id_to_wd = {wd_to_id[wd]: wd for wd in wd_to_id.keys()}
id_to_lb = {lb_to_id[lb]: lb for lb in lb_to_id.keys()}
for i in range(10):
    print(list(wd_ary)[i], list(wd_cnt)[i])
    #print(list(wd_set.keys())[i], list(wd_set.values())[i])

EOS 13622
ます 3892
た 3525
し 3275
て 3200
の 3012
です 2875
か 2711
！ 2416
？ 2338


# 重要！！

In [34]:
KIND = len(list(lb_to_id.keys()))
CUT_OFF = 2

print("words kinds:",
      len(wd_cnt), "words>=" + str(CUT_OFF) + ":", np.sum(wd_cnt >= CUT_OFF))
print("all words num:", np.sum(wd_cnt))
print("all words num:", np.sum(wd_cnt[wd_cnt >= CUT_OFF]))

other_id = np.sum(wd_cnt >= CUT_OFF)
wd_to_id.update({wd: other_id for wd in wd_ary[wd_cnt < CUT_OFF]})
print(other_id)
id_to_wd[other_id] = '[X]'

print(max(wd_to_id.values()))

with open('data/Coordination_A_id_to_wd.pickle', mode='wb') as f:
    pickle.dump(id_to_wd, f)
with open('data/Coordination_A_wd_to_id.pickle', mode='wb') as f:
    pickle.dump(wd_to_id, f)
with open('data/Coordination_A_wd_set.pickle', mode='wb') as f:
    pickle.dump(wd_set, f)
with open('data/Coordination_A_id_to_lb.pickle', mode='wb') as f:
    pickle.dump(id_to_lb, f)
pd.DataFrame(
    np.array(list(id_to_lb.items())).T, index=["ラベルid", "ラベルname"])

words kinds: 2976 words>=2: 2908
all words num: 120631
all words num: 120563
2908
2908


,0,1,2,3,4,5
ラベルid,0,1,2,3,4,5
ラベルname,technical coordination,time management,proceedings,blank,task division,quote


In [35]:
of = open("data/Coordination_A_edu_data.txt", "w")
for i, (sen, lb) in enumerate(zip(np.array(doc), input_data[:, label])):
    sen_str = "".join([str(wd_to_id[wd]) + " " for wd in sen])
    print_str = str(lb_to_id[lb]) + " " + str(lb_to_id[lb]) + " " + sen_str
    print(print_str, file=of)
of.close()

of = open("data/Coordination_A_edu_data_pre.txt", "w")
for i, sen in enumerate(np.array(doc_pre)):
    sen_str = "".join([str(wd_to_id[wd]) + " " for wd in sen])
    print(sen_str, file=of)
of.close()

print(max([len(sen) for sen in doc]))

print(KIND)

313
6


# train